## About this notebook
In this notebook i am going to test 5 classification techniques in predicting fraud cases in a unbalanced dataset. The 5 ways are Logistic Regression,KNearest, Support Vector Classificator,Decision Tree Classifier and XGBoost.


In [24]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import xgboost as xgb

# Classifier Libraries
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier


# Other Libraries
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

import warnings
warnings.filterwarnings("ignore")

I had to balance a sample of the dataset to get better models.The original dataset have a majority of non-frauds. With an unbalanced dataset a "dumb" model will get andvantage by tagging every row as a non-fraud and get a good accuracy score. In the balanced dataset i got all the fraud cases and shuffle them between the same number of non-fraud cases, so the new dataset have 492 + 492 = 984 cases in total.

In [25]:

#importing balanced data
data = pd.read_csv('C:\\Users\\User\\Documents\\ML\\simulaçoes de portfolio\\fraud\\balanced_data.csv')

Here I prepared the data for the models. I separed the data into training data and testing data. Also the XGBoost method requires some preparation for its data too.

In [26]:

X,y = data.drop(['Class'],axis=1) , data[['Class']]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train=np.array(X_train)
X_test=np.array(X_test)
y_train= np.array(y_train)
y_test=np.array(y_test)

data_dmatrix = xgb.DMatrix(data=X,label=y)

xg_reg = xgb.XGBClassifier(colsample_bytree = 0.3, learning_rate = 0.3,
                max_depth = 10, alpha = 15, n_estimators = 10, eval_metric='logloss')

Defining the methods:

In [27]:
#simple classifiers
classifiers = {
    "LogisiticRegression": LogisticRegression(),
    "KNearest": KNeighborsClassifier(),
    "Support Vector Classifier": SVC(),
    "DecisionTreeClassifier": DecisionTreeClassifier(),
    "XGBoost":xg_reg
}


Training and testing the models.

In [28]:


training_score_f1=[]
training_score_recall=[]

for key, classifier in classifiers.items():
    classifier.fit(X_train, y_train)
    training_score_f1.append(np.mean(cross_val_score(classifier, X_test, y_test, cv=10,scoring='f1')))
    training_score_recall.append(np.mean(cross_val_score(classifier, X_test, y_test, cv=10,scoring='recall')))
    

Now we need to see how the models perform. For this problem the f1 score and recall are good metrics and here is why: For fraud detection we really don't want a model that accuses non-fraud cases as fraudulent. The recall metric is calculated as (true positives)/(true positives + false negatives), wich penalises a model that perform with a lot of false negatives by giving a low score. The f1-score is calculated as (true positive)/(true positive +2(false positives + false negatives)) and also penalises models that make a lot of mistakes. 

In [29]:
perff1={"LogisiticRegression":training_score_f1[0],"KNearest":training_score_f1[1],"Support Vector Classifier":training_score_f1[2],"DecisionTreeClassifier":training_score_f1[3],"XGBoost":training_score_f1[4]}
perfre={"LogisiticRegression":training_score_recall[0],"KNearest":training_score_recall[1],"Support Vector Classifier":training_score_recall[2],"DecisionTreeClassifier":training_score_recall[3],"XGBoost":training_score_recall[4]}
    
    
perf=pd.DataFrame([perff1,perfre]).rename(index={0:'f1 score',1:'recall'})
perf.to_csv('performace.csv')
perf

,LogisiticRegression,KNearest,Support Vector Classifier,DecisionTreeClassifier,XGBoost
f1 score,0.905019,0.622881,0.651476,0.878436,0.913825
recall,0.867273,0.659091,0.912727,0.912727,0.868182


We can see that Logistic Regression, Decision Tree and XGBoost have a good testing score